In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# Initialize Spark session
spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/12/03 00:52:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
os.listdir('/standard/ds7200-apt4c/isaac_yuyang_final_project/')

['flights_2022.parquet',
 'flights_2021.parquet',
 'flights_2020.parquet',
 'flights_2018.parquet',
 'flights_2019.parquet',
 'Airlines.csv']

In [4]:
airlines = spark.read.load('/standard/ds7200-apt4c/isaac_yuyang_final_project/Airlines.csv', header=True, format='csv', sep=',')

In [5]:
airlines.show(10)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|       Titan Airways|
| 04Q|  Tradewind Aviation|
| 05Q| Comlux Aviation, AG|
| 06Q|Master Top Linhas...|
| 07Q| Flair Airlines Ltd.|
| 09Q|      Swift Air, LLC|
| 0BQ|                 DCA|
| 0CQ|ACM AIR CHARTER GmbH|
| 0GQ|Inter Island Airw...|
| 0HQ|Polar Airlines de...|
+----+--------------------+
only showing top 10 rows



In [7]:
airlines.count()

1571

In [3]:
# Read in data
flights_2018 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2018.parquet')
flights_2019 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2019.parquet')
flights_2020 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2020.parquet')
flights_2021 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2021.parquet')
flights_2022 = spark.read.parquet('/standard/ds7200-apt4c/isaac_yuyang_final_project/flights_2022.parquet')

In [9]:
flights_2018.columns

['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRS

In [30]:
flights_2018.groupBy('OriginCityName').count().show()

+------------------+-----+
|    OriginCityName|count|
+------------------+-----+
|   Gainesville, FL| 2092|
|      Richmond, VA|18967|
|       Ontario, CA|18590|
|        Tucson, AZ|13103|
|     Pago Pago, TT|  122|
|  Myrtle Beach, SC| 8961|
|       Medford, OR| 6696|
|  Palm Springs, CA| 8041|
|       Redding, CA| 1169|
|     St. Cloud, MN|  145|
|       Durango, CO| 2851|
|   Devils Lake, ND|  626|
|      Gustavus, AK|   31|
|        Mobile, AL| 4027|
|Corpus Christi, TX| 4291|
|       Dubuque, IA|  435|
|      Columbus, GA|  606|
|    Huntsville, AL| 6156|
|    Fort Myers, FL|25611|
|     Pensacola, FL| 7248|
+------------------+-----+
only showing top 20 rows



In [4]:
# Combine all years
combined = flights_2018.union(flights_2019).union(flights_2020).union(flights_2021).union(flights_2022)

In [14]:
combined.select('ArrDelay', 'ArrDelayMinutes').show(20)

+--------+---------------+
|ArrDelay|ArrDelayMinutes|
+--------+---------------+
|    -8.0|            0.0|
|    -6.0|            0.0|
|    -2.0|            0.0|
|   -11.0|            0.0|
|    -1.0|            0.0|
|    22.0|           22.0|
|    -1.0|            0.0|
|    -9.0|            0.0|
|    null|           null|
|    22.0|           22.0|
|   -12.0|            0.0|
|   -13.0|            0.0|
|   -10.0|            0.0|
|   -13.0|            0.0|
|    16.0|           16.0|
|   105.0|          105.0|
|   -11.0|            0.0|
|     8.0|            8.0|
|     1.0|            1.0|
|   -10.0|            0.0|
+--------+---------------+
only showing top 20 rows



In [16]:
numeric_cols = [field.name for field in combined.schema.fields 
                if field.dataType.typeName() in ['integer', 'double', 'float', 'long']]
correlations = []
for col in numeric_cols:
    if col not in ['ArrDelay', 'ArrDelayMinutes', 'ArrivalDelayGroups', 'ArrDel15']:
        corr = combined.stat.corr(col, 'ArrDelay')
        correlations.append((col, corr))
# Sort by absolute correlation value
correlations.sort(key=lambda x: abs(x[1]), reverse=True)

# Display results
for col, corr in correlations:
    print(f"{col}: {corr:.4f}")

DepDelay: 0.9572
DepDelayMinutes: 0.9540
DepartureDelayGroups: 0.8092
DepDel15: 0.5540
TaxiOut: 0.2005
DepTime: 0.1128
WheelsOff: 0.1092
TaxiIn: 0.1055
CRSDepTime: 0.0826
CRSArrTime: 0.0690
ActualElapsedTime: 0.0465
WheelsOn: 0.0350
DOT_ID_Marketing_Airline: 0.0340
ArrTime: 0.0287
DOT_ID_Operating_Airline: 0.0266
__index_level_0__: 0.0181
DestWac: -0.0128
AirTime: 0.0116
CRSElapsedTime: -0.0106
Flight_Number_Operating_Airline: 0.0089
Flight_Number_Marketing_Airline: 0.0089
OriginAirportSeqID: -0.0081
OriginAirportID: -0.0081
OriginCityMarketID: -0.0080
DistanceGroup: -0.0072
DestStateFips: 0.0071
Distance: -0.0071
OriginWac: -0.0071
DayOfWeek: 0.0048
Year: -0.0043
Quarter: 0.0042
Month: 0.0042
OriginStateFips: 0.0023
DivAirportLandings: -0.0022
DayofMonth: -0.0020
DestAirportSeqID: -0.0017
DestAirportID: -0.0017
DestCityMarketID: 0.0000


It looks like the only variables that are very strongly correlated with arrival delay are variables related to departure delay, which makes sense, but isn't particularly useful for predicting flight status, since if it's delayed departing then it isn't suprising that it is delayed arriving.

`TaxiOut`, which is the time in minutes that it took to taxi out from the departure gate, has a correlation of 0.2, which shows it could maybe be a useful predictor.

In [26]:
# Group by airline and date, calculate proportion of delayed flights
delay_prop = combined.groupBy('Airline') \
                .agg(F.mean('ArrDel15').alias('delay_proportion')) \
                .select('Airline', 'delay_proportion')

In [23]:
delay_rate = delay_prop \
    .withColumn('Delay Rate', F.round(F.col('delay_proportion') * 100, 2)) \
    .select('Airline', 'Delay Rate') \
    .orderBy('Delay Rate')

In [24]:
delay_rate.show(30, truncate=False)

[Stage 26:====================================================>   (28 + 2) / 30]

+-----------------------------------------+----------+
|Airline                                  |Delay Rate|
+-----------------------------------------+----------+
|Cape Air                                 |8.94      |
|Hawaiian Airlines Inc.                   |11.0      |
|Endeavor Air Inc.                        |12.99     |
|Delta Air Lines Inc.                     |13.05     |
|Horizon Air                              |14.09     |
|Empire Airlines Inc.                     |15.78     |
|Capital Cargo International              |16.19     |
|SkyWest Airlines Inc.                    |16.3      |
|Republic Airlines                        |16.43     |
|Alaska Airlines Inc.                     |16.78     |
|Envoy Air                                |17.39     |
|Comair Inc.                              |17.57     |
|Southwest Airlines Co.                   |18.11     |
|Air Wisconsin Airlines Corp              |18.15     |
|Compass Airlines                         |18.17     |
|United Ai

In [32]:
quantiles = delay_prop.approxQuantile('delay_proportion', [0.25, 0.75], 0.01)
q1, q3 = quantiles[0], quantiles[1]
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

filtered = delay_prop.filter((F.col('delay_proportion') >= lower_bound) & (F.col('delay_proportion') <= upper_bound)) \
                    .withColumn('Delay Rate', F.round(F.col('delay_proportion') * 100, 2)) \
                    .select('Airline', 'Delay Rate') \
                    .orderBy('Delay Rate')

filtered.show(30, truncate=False)

[Stage 44:======================================================> (29 + 1) / 30]

+----------------------------------------+----------+
|Airline                                 |Delay Rate|
+----------------------------------------+----------+
|Hawaiian Airlines Inc.                  |11.0      |
|Endeavor Air Inc.                       |12.99     |
|Delta Air Lines Inc.                    |13.05     |
|Horizon Air                             |14.09     |
|Empire Airlines Inc.                    |15.78     |
|Capital Cargo International             |16.19     |
|SkyWest Airlines Inc.                   |16.3      |
|Republic Airlines                       |16.43     |
|Alaska Airlines Inc.                    |16.78     |
|Envoy Air                               |17.39     |
|Comair Inc.                             |17.57     |
|Southwest Airlines Co.                  |18.11     |
|Air Wisconsin Airlines Corp             |18.15     |
|Compass Airlines                        |18.17     |
|United Air Lines Inc.                   |18.53     |
|American Airlines Inc.     

By grouping by airline and taking the mean of the delayed arrivals, we can find the proportion of flights for each airline that are delayed by at least 15 minutes. Some outliers appear to be Cape Air with 8.9%, Endeavor Airlines with 12.9%, and Peninsula, with  37%. Most of the rest of the airlines are somewhere around 20%, $\pm$ ~6%.

In [37]:
q1-1.5*iqr

0.10403509952998055

In [36]:
q3+1.5*iqr

0.2584557548334021